In [33]:
import pandas as pd
import numpy as np

In [34]:
df_credits = pd.read_csv('../data/crudo/credits.csv')

In [35]:
df_credits.dtypes

cast    object
crew    object
id       int64
dtype: object

In [36]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [37]:
df_credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

Función para desanidar la columna 'cast'

In [38]:
import ast

def extract_cast(data):
    cast_list = []
    for idx, row in data.iterrows():
        cast = ast.literal_eval(row['cast'])
        for c in cast:
            cast_list.append({
                'id': row['id'],
                'cast_id': c.get('cast_id'),
                'character': c.get('character'),
                'credit_id': c.get('credit_id'),
                'gender': c.get('gender'),
                'name': c.get('name'),
                'order': c.get('order')
            })
    return pd.DataFrame(cast_list)

In [39]:
# Extraje la columna 'cast' en un DataFrame separado
cast_df = extract_cast(df_credits)

In [40]:
cast_df.head()

,id,cast_id,character,credit_id,gender,name,order
0,862,14,Woody (voice),52fe4284c3a36847f8024f95,2,Tom Hanks,0
1,862,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,Tim Allen,1
2,862,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,Don Rickles,2
3,862,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,Jim Varney,3
4,862,18,Rex (voice),52fe4284c3a36847f8024fa5,2,Wallace Shawn,4


Función para desanidar la columna 'crew'

In [41]:
def extract_crew(data):
    crew_list = []
    for idx, row in data.iterrows():
        crew = ast.literal_eval(row['crew'])
        for c in crew:
            crew_list.append({
                'id': row['id'],
                'credit_id': c.get('credit_id'),
                'department': c.get('department'),
                'gender': c.get('gender'),
                'job': c.get('job'),
                'name': c.get('name')
            })
    return pd.DataFrame(crew_list)

In [42]:
# Extraje la columna 'crew' en un DataFrame separado
crew_df = extract_crew(df_credits)

In [95]:
crew_df.head()

,id,credit_id,department,gender,job,name
0,862,52fe4284c3a36847f8024f49,Directing,2,Director,John Lasseter
1,862,52fe4284c3a36847f8024f4f,Writing,2,Screenplay,Joss Whedon
2,862,52fe4284c3a36847f8024f55,Writing,2,Screenplay,Andrew Stanton
3,862,52fe4284c3a36847f8024f5b,Writing,2,Screenplay,Joel Cohen
4,862,52fe4284c3a36847f8024f61,Writing,0,Screenplay,Alec Sokolow


Verifico si hay datos nulos

In [96]:
cast_df.isnull().sum()

id           0
cast_id      0
character    0
credit_id    0
gender       0
name         0
order        0
dtype: int64

In [97]:
crew_df.isnull().sum()

id            0
credit_id     0
department    0
gender        0
job           0
name          0
dtype: int64

Verifico y elimino filas duplicadas de cada columna

In [98]:
print(f'Tamaño del set antes de eliminar las filas repetidas: {cast_df.shape}')
cast_df.drop_duplicates(inplace=True)
print(f'Tamaño del set despues de eliminar las filas repetidas: {cast_df.shape}')

Tamaño del set antes de eliminar las filas repetidas: (562044, 7)
Tamaño del set despues de eliminar las filas repetidas: (562044, 7)


In [99]:
print(f'Tamaño del set antes de eliminar las filas repetidas: {crew_df.shape}')
crew_df.drop_duplicates(inplace=True)
print(f'Tamaño del set despues de eliminar las filas repetidas: {crew_df.shape}')

Tamaño del set antes de eliminar las filas repetidas: (463836, 6)
Tamaño del set despues de eliminar las filas repetidas: (463836, 6)


Exporto los DataFrames a archivos CSV separados

In [48]:
cast_df.to_csv('cast.csv', index=False)
crew_df.to_csv('crew.csv', index=False)

Uno los DataFrames con el DataFrame principal

In [49]:
df_credits_merged = df_credits.drop(columns=['cast', 'crew'])
df_credits_merged = df_credits_merged.merge(cast_df, on='id', how='left')
df_credits_merged = df_credits_merged.merge(crew_df, on='id', how='left')

In [50]:
df_credits_merged.head()

,id,cast_id,character,credit_id_x,gender_x,name_x,order,credit_id_y,department,gender_y,job,name_y
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,Tom Hanks,0.0,52fe4284c3a36847f8024f49,Directing,2.0,Director,John Lasseter
1,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,Tom Hanks,0.0,52fe4284c3a36847f8024f4f,Writing,2.0,Screenplay,Joss Whedon
2,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,Tom Hanks,0.0,52fe4284c3a36847f8024f55,Writing,2.0,Screenplay,Andrew Stanton
3,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,Tom Hanks,0.0,52fe4284c3a36847f8024f5b,Writing,2.0,Screenplay,Joel Cohen
4,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,Tom Hanks,0.0,52fe4284c3a36847f8024f61,Writing,0.0,Screenplay,Alec Sokolow


In [51]:
df_credits_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9484668 entries, 0 to 9484667
Data columns (total 12 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   cast_id      float64
 2   character    object 
 3   credit_id_x  object 
 4   gender_x     float64
 5   name_x       object 
 6   order        float64
 7   credit_id_y  object 
 8   department   object 
 9   gender_y     float64
 10  job          object 
 11  name_y       object 
dtypes: float64(4), int64(1), object(7)
memory usage: 868.3+ MB


Defino y elimino columnas innecesarias

In [52]:
col_innecesarias = ['cast_id', 'character', 'credit_id_x', 'gender_x', 'order', 'credit_id_y', 'gender_y']
df_credits_merged = df_credits_merged.drop(columns=col_innecesarias)

In [100]:
df_credits_merged.head()

,id,actor,department,job,director
0,862,Tom Hanks,Directing,Director,John Lasseter
1,862,Tom Hanks,Writing,Screenplay,Joss Whedon
2,862,Tom Hanks,Writing,Screenplay,Andrew Stanton
3,862,Tom Hanks,Writing,Screenplay,Joel Cohen
4,862,Tom Hanks,Writing,Screenplay,Alec Sokolow


In [101]:
df_credits_merged.shape

(9484668, 5)

In [59]:
# Modifico los nombres de las columnas
column_rename_map = {
    'id': 'id',
    'name_x': 'actor',
    'department': 'department',
    'job': 'job',
    'name_y': 'director'
}

# Cambio los nombres de las columnas
df_credits_merged.rename(columns=column_rename_map, inplace=True)

In [127]:
df_credits_merged.head()

,id,actor,department,job,director
0,862,Tom Hanks,Directing,Director,John Lasseter
1,862,Tom Hanks,Writing,Screenplay,Joss Whedon
2,862,Tom Hanks,Writing,Screenplay,Andrew Stanton
3,862,Tom Hanks,Writing,Screenplay,Joel Cohen
4,862,Tom Hanks,Writing,Screenplay,Alec Sokolow


Verifico categorias unicas

In [128]:
print("Categorías únicas en 'actor':", df_credits_merged['actor'].nunique())
print("Categorías únicas en 'department':", df_credits_merged['department'].nunique())
print("Categorías únicas en 'job':", df_credits_merged['job'].nunique())
print("Categorías únicas en 'director':", df_credits_merged['director'].nunique())

Categorías únicas en 'actor': 202747
Categorías únicas en 'department': 12
Categorías únicas en 'job': 456
Categorías únicas en 'director': 157614


In [133]:
df_credits_merged = df_credits_merged.dropna(subset=['id', 'actor', 'department', 'job', 'director'])

In [134]:
print("Categorías únicas en 'actor':", df_credits_merged['actor'].nunique())
print("Categorías únicas en 'department':", df_credits_merged['department'].nunique())
print("Categorías únicas en 'job':", df_credits_merged['job'].nunique())
print("Categorías únicas en 'director':", df_credits_merged['director'].nunique())

Categorías únicas en 'actor': 202299
Categorías únicas en 'department': 12
Categorías únicas en 'job': 456
Categorías únicas en 'director': 155784


Trabajo los valores nulos `NaN` y 'Null'

In [135]:
df_credits_merged = df_credits_merged.dropna(how='all')

Verifico si quedaron filas con 'NaN'

In [140]:
df_credits_merged.isna().sum()

id            0
actor         0
department    0
job           0
director      0
dtype: int64

In [143]:
print(f'Tamaño del set antes de eliminar las filas repetidas: {df_credits_merged.shape}')
df_credits_merged.drop_duplicates(inplace=True)
print(f'Tamaño del set despues de eliminar las filas repetidas: {df_credits_merged.shape}')

Tamaño del set antes de eliminar las filas repetidas: (9477663, 5)
Tamaño del set despues de eliminar las filas repetidas: (9450206, 5)


In [159]:
df_credits_merged.fillna(0, inplace=True)

In [160]:
df_credits_merged.shape

(9450206, 5)

Exporto el archivo limpio

In [163]:
# Exporto en archivo Parquet
df_credits_merged.to_parquet('../data/procesado/credits_pilabs.parquet', engine='pyarrow')